<a href="https://colab.research.google.com/github/InowaR/colab/blob/main/ai_telebot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install telebot

In [31]:
import sys

# Additional authentication is required for Google Colab
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

In [44]:
from google.colab import userdata
import vertexai
from vertexai.generative_models import GenerationConfig, GenerativeModel, Image, Part

# Define project information
PROJECT_ID = userdata.get('PROJECT_ID')
LOCATION = userdata.get('LOCATION')


# create a new project
# https://console.cloud.google.com/vertex-ai/


vertexai.init(project=PROJECT_ID, location=LOCATION)

multimodal_model = GenerativeModel("gemini-1.0-pro-vision")

In [45]:
import http.client
import typing
import urllib.request


def get_image_bytes_from_url(image_url: str) -> bytes:
    with urllib.request.urlopen(image_url) as response:
        response = typing.cast(http.client.HTTPResponse, response)
        image_bytes = response.read()
    return image_bytes

def load_image_from_url(image_url: str) -> Image:
    image_bytes = get_image_bytes_from_url(image_url)
    return Image.from_bytes(image_bytes)

def send_data(image_url, prompt):
    try:
        image = load_image_from_url(image_url)
        contents = [image, prompt]
        responses = multimodal_model.generate_content(contents, stream=True)
        answer = ''
        for response in responses:
            answer += response.text
        return answer
    except:
        pass

In [80]:
import requests
import telebot
from telebot.types import ReplyKeyboardMarkup, KeyboardButton
from google.colab import userdata


BOT_TOKEN = userdata.get('TELEGRAM')

bot = telebot.TeleBot(BOT_TOKEN)

data = {}


@bot.message_handler(commands=['start'])
def handle_start(message):
    keyboard = ReplyKeyboardMarkup(resize_keyboard=True)
    keyboard.add(KeyboardButton('Удалить текст'))
    keyboard.add(KeyboardButton('Удалить изображение и текст'))
    bot.send_message(message.chat.id, f"1) Выберите изображение и нажмите отправить.\n2) Напишите текст промпта и нажмите отправить.\n3) Ожидайте ответ.", reply_markup=keyboard)


@bot.message_handler(func=lambda message: message.text == 'Удалить текст')
def clear_text(message):
    global data
    if 'text' in data:
        del data['text']
    bot.send_message(message.chat.id, f'Текст удален')


@bot.message_handler(func=lambda message: message.text == 'Удалить изображение и текст')
def clear_all(message):
    global data
    data = {}
    bot.send_message(message.chat.id, f"Изображение и текст удалены")


@bot.message_handler(content_types=['text'])
def send_text(message):
    if 'text' in data:
        existing_line = data['text']
        new_line = existing_line + '. ' + message.text
        data['text'] = new_line
        if 'image_url' in data:
            bot.send_message(message.chat.id, f"Данные отправлены, ожидайте ответ")
            answer = send_data(image_url=data['image_url'], prompt=data['text'])
            bot.send_message(message.chat.id, f"{answer}")
    elif 'text' not in data:
        data['text'] = message.text
        bot.send_message(message.chat.id, f"Текст добавлен")
        if 'image_url' in data:
            bot.send_message(message.chat.id, f"Данные отправлены, ожидайте ответ")
            answer = send_data(image_url=data['image_url'], prompt=data['text'])
            bot.send_message(message.chat.id, f"{answer}")
    elif 'image_url' not in data:
        bot.send_message(message.chat.id, f"Текст уже отправлен, добавьте фотографию")
    elif 'image_url' in data and 'text' in data:
        bot.send_message(message.chat.id, f"Данные отправлены, ожидайте ответ")
        answer = send_data(image_url=data['image_url'], prompt=data['text'])
        bot.send_message(message.chat.id, f"{answer}")



@bot.message_handler(content_types=['photo'])
def send_photo(message):
    photo = message.photo[-1]
    file_id = photo.file_id
    file_info = bot.get_file(file_id)
    download_url = f"https://api.telegram.org/file/bot{BOT_TOKEN}/{file_info.file_path}"

    if 'image_url' not in data:
        data['image_url'] = download_url
        bot.send_message(message.chat.id, f"Изображение добавлено")
        if 'text' in data:
            bot.send_message(message.chat.id, f"Данные отправлены, ожидайте ответ")
            answer = send_data(image_url=data['image_url'], prompt=data['text'])
            bot.send_message(message.chat.id, f"{answer}")
    elif 'text' not in data:
        bot.send_message(message.chat.id, f"Изображение уже добавлено, добавьте текст")
    elif 'image_url' in data and 'text' in data:
        bot.send_message(message.chat.id, f"Данные отправлены, ожидайте ответ")
        answer = send_data(image_url=data['image_url'], prompt=data['text'])
        bot.send_message(message.chat.id, f"{answer}")


bot.polling()
